# **Prompt Engineering**

##### _ChatGPT Prompt Engineering for Developers_

## **_Principles_** ##

### **Instructions** ####

__Notes on using the OpenAI API outside of this classroom__
    
To install the OpenAI Python library - 
    
    !pip install openai
    
The library needs to be configured with your account's secret key, which is available on the website.

You can either set it as the OPENAI_API_KEY environment variable before using the library: - 
    
    !export OPENAI_API_KEY='sk-...'

Or, set openai.api_key to its value:

    import openai
    openai.api_key = "sk-..."

A note about the backslash:

_In the course, we are using a __backslash '\\'__ to make the text fit on the screen without inserting __newline '\n'__ characters. GPT-3 isn't really affected whether you insert newline characters or not. But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance._

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### **Principle 1** ###

##### Write clear and specific instructions

__Tactic 1__: Use delimiters to clearly indicate distinct parts of the input. 

Delimiters can be anything like:

    ```, """, < >, <tag> </tag>, :

In [ ]:
text = f"""You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. This will guide the model towards the desired output, and reduce the chances 
of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed 
and relevant outputs."""

prompt = f"""Summarize the text delimited by triple backticks into a single sentence. ```{text}``` """

response = get_completion(prompt)
print(response)

__Output Response__

    To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which can be achieved through longer prompts that offer more clarity and context.

__Tactic 2__: Ask for a structured output : __JSON, HTML__

In [ ]:
prompt = f"""Generate a list of three made-up book titles along with their authors and genres. Provide them in JSON format with the following keys: book_id, title, author, genre."""

response = get_completion(prompt)
print(response)

__Output Response__

    {
      "books": [
        {
          "book_id": 1,
          "title": "The Enigma of Elysium",
          "author": "Evelyn Sinclair",
          "genre": "Mystery"
        },
        {
          "book_id": 2,
          "title": "Whispers in the Wind",
          "author": "Nathaniel Blackwood",
          "genre": "Fantasy"
        },
        {
          "book_id": 3,
          "title": "Echoes of the Past",
          "author": "Amelia Hart",
          "genre": "Romance"
        }
      ]
    }

__Tactic 3__: Ask the model to check whether conditions are satisfied

In [ ]:
text_1 = f"""Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. Let it sit for a 
bit so the tea can steep. After a few minutes, take out the tea bag. If you like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious cup of tea to enjoy."""

prompt = f"""
You will be provided with text delimited by triple quotes. 

If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt)

print("Completion for Text 1:")
print(response)

__Output Response__

    Completion for Text 1:
    Step 1 - Get some water boiling.
    Step 2 - Grab a cup and put a tea bag in it.
    Step 3 - Once the water is hot enough, pour it over the tea bag.
    Step 4 - Let it sit for a bit so the tea can steep.
    Step 5 - After a few minutes, take out the tea bag.
    Step 6 - If you like, add some sugar or milk to taste.
    Step 7 - Enjoy your delicious cup of tea.

In [ ]:
text_2 = f"""The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a walk in the park. The flowers are blooming, and the trees are swaying gently in the breeze. People are out and about, 
enjoying the lovely weather. Some are having picnics, while others are playing games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the beauty of nature."""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

response = get_completion(prompt)

print("Completion for Text 2:")
print(response)

__Output__

    Completion for Text 2:
    No steps provided.

__Tactic 4__: _"Few-shot"_ prompting

In [ ]:
prompt = f"""Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

response = get_completion(prompt)

print(response)

__Output__

    <grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering spirit that rises again and again, even in the face of adversity. Just as a phoenix emerges from the ashes, resilience allows us to bounce back and grow stronger from life's challenges.

### **Principle 2** ###

##### Give the model time to “think”

__Tactic 1__: Specify the steps required to complete a task

In [ ]:
text = f"""In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill 
following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight."""

prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Separate your answers with line breaks.

Text:```{text}``` 
"""

response = get_completion(prompt_1)

print("Completion for prompt 1:")
print(response)

__Output__

    Completion for prompt 1:
    1 - Jack and Jill, siblings, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit, yet they return home and remain 
    undeterred in their adventurous spirits.

    2 - Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant ils rentrent 
    chez eux et restent déterminés dans leur esprit d'aventure.

    3 - Jack, Jill

    4 - {
    "french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par Jill, pourtant 
                        ils rentrent chez eux et restent déterminés dans leur esprit d'aventure.",
    "num_names": 2
    }

    Ask for output in a specified format

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 

1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:

Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

response = get_completion(prompt_2)

print("\nCompletion for prompt 2:")
print(response)

__Output__

    Completion for prompt 2:
    Summary: Jack and Jill, siblings from a charming village, go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips on a stone and tumbles down the hill, with Jill following suit. 
    Despite the mishap, they return home and continue exploring with delight.

    Translation: Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, suivi par 
    Jill. Malgré l'accident, ils rentrent chez eux et continuent à explorer avec joie.

    Names: Jack, Jill

    Output JSON: 
    {
    "french_summary": "Jack et Jill, frère et sœur d'un charmant village, partent en quête d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche sur une pierre et dévale la colline, 
                        suivi par Jill. Malgré l'accident, ils rentrent chez eux et continuent à explorer avec joie.",
    "num_names": 2
    }

__Tactic 2__: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot

What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_completion(prompt)

print(response)

''' Note that the student's solution is actually not correct. We can fix this by instructing the model to work out its own solution first.'''

__Output__

    The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.

In [ ]:
prompt = f"""
Your task is to determine if the student's solution is correct or not. To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not. Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

response = get_completion(prompt)

print(response)

__Output__

    To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

    Let x be the size of the installation in square feet.

    Costs:
    1. Land cost: $100 * x
    2. Solar panel cost: $250 * x
    3. Maintenance cost: $100,000 + $10 * x

    Total cost: $100 * x + $250 * x + $100,000 + $10 * x = $360 * x + $100,000

    Is the student's solution the same as the actual solution just calculated:
    No

    Student grade:
    Incorrect

### **Model Limitations: Hallucinations**

_**Boie** is a real company, the product name is not real._

In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie.
"""

response = get_completion(prompt)

print(response)

__Output__

    The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide a superior brushing experience. Boie is a company known for its innovative oral care products, and the 
    AeroGlide UltraSlim Smart Toothbrush is no exception.

    One of the standout features of this toothbrush is its ultra-slim design. The brush head is only 2mm thick, making it much thinner than traditional toothbrushes. This slim profile allows for better access to 
    hard-to-reach areas of the mouth, ensuring a thorough and effective clean.

    The AeroGlide UltraSlim Smart Toothbrush also incorporates smart technology. It connects to a mobile app via Bluetooth, allowing users to track their brushing habits and receive personalized recommendations for 
    improving their oral hygiene routine. The app provides real-time feedback on brushing technique, duration, and coverage, helping users to achieve optimal oral health.

    The toothbrush features soft, antimicrobial bristles made from a durable thermoplastic elastomer. These bristles are gentle on the gums and teeth, while also being effective at removing plaque and debris. The 
    antimicrobial properties of the bristles help to inhibit the growth of bacteria, keeping the brush clean and hygienic.

    In terms of battery life, the AeroGlide UltraSlim Smart Toothbrush boasts an impressive 30-day battery life on a single charge. This makes it convenient for travel and ensures that users don't have to worry about 
    constantly recharging their toothbrush.

    Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie offers a combination of advanced technology, slim design, and effective cleaning capabilities. It is a great option for those looking to upgrade their oral care 
    routine and achieve a healthier smile.

## **_Iterative Prompt Development_** ##

##### In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

### **Setup** ###

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### **Generate a marketing product description from a product fact sheet** ###

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:  medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.

SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

__Output__

    Introducing our stunning mid-century inspired office chair, the perfect addition to any home or business setting. This chair is part of a beautiful family of office furniture, including filing cabinets, desks, 
    bookcases, meeting tables, and more, all designed with a timeless mid-century aesthetic.

    One of the standout features of this chair is the variety of customization options available. You can choose from several shell colors and base finishes to perfectly match your existing decor. The chair is available 
    with either plastic back and front upholstery or full upholstery in a range of 10 fabric and 6 leather options, allowing you to create a look that is uniquely yours.

    The chair is also available with or without armrests, giving you the flexibility to choose the option that best suits your needs. The base finish options include stainless steel, matte black, gloss white, or chrome, 
    ensuring that you can find the perfect match for your space.

    In terms of construction, this chair is built to last. It features a 5-wheel plastic coated aluminum base, providing stability and mobility. The pneumatic chair adjust allows for easy raise and lower action, ensuring 
    optimal comfort throughout the day.

    When it comes to dimensions, this chair is designed with both style and comfort in mind. With a width of 53 cm (20.87"), depth of 51 cm (20.08"), and height of 80 cm (31.50"), it offers ample space without overwhelming 
    your space. The seat height is 44 cm (17.32") and the seat depth is 41 cm (16.14"), providing a comfortable seating experience for users of all heights.

    We understand that every space is unique, which is why we offer a range of options to further customize your chair. You can choose between soft or hard-floor caster options, ensuring that your chair glides smoothly across 
    any surface. Additionally, you have the choice between two seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3), allowing you to select the level of support that suits your preferences. The chair is also available 
    with armless design or 8 position PU armrests, providing additional comfort and versatility.

    When it comes to materials, this chair is crafted with the utmost attention to quality. The shell base glider is made from cast aluminum with a modified nylon PA6/PA66 coating, ensuring durability and longevity. The shell 
    thickness is 10 mm, providing a sturdy and reliable structure. The seat is made from HD36 foam, offering a comfortable and supportive seating experience.

    Finally, this chair is proudly made in Italy, known for its exceptional craftsmanship and design. With its combination of style, functionality, and customization options, this mid-century inspired office chair is the 
    perfect choice for those seeking a timeless and elegant addition to their workspace.

    Please note that this chair is qualified for contract use, making it suitable for a wide range of professional settings. Upgrade your office or home with this exceptional piece of furniture and experience the perfect blend 
    of style and comfort.

### **Issue 1: The text is too long** ###

Limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(f"response:\n\n{response}\nResponse Length:\n\n{len(response)}")

__Output__

    Response: 

    Introducing our mid-century inspired office chair, part of a stunning furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a durable aluminum base with 5 wheels and pneumatic height adjustment. Perfect for home or business use. Made in Italy.

    Response Length: 

    333

### **Issue 2. Text focuses on the wrong details** ###

Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

__Output__

    Introducing our mid-century inspired office chair, part of a beautiful furniture collection. With various shell colors and base finishes, it offers versatility for any setting. Choose between plastic or full upholstery 
    in a range of fabric and leather options. The chair features a durable aluminum base and pneumatic chair adjustment for easy height customization. Made in Italy.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

__Output__

    Introducing our mid-century inspired office chair, part of a beautiful family of furniture. With various shell colors and base finishes, this chair offers versatility and style. Choose between plastic or full upholstery in a range of fabric and leather options. The chair features a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy height adjustment. Available with or without armrests, this chair is suitable for both home and business settings. Made with high-quality materials, including a cast aluminum shell and HD36 foam seat, this chair is built to last. Product ID: SWC-100, SWC-110.

### **Issue 3. Description needs a table of dimensions**

Ask it to extract information and organize it in a table.

In [ ]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

In [3]:
# Repsonse Output 

from IPython.display import display, HTML

response = '''
<div>
  <h2>Product Description</h2>
  <p>
    Introducing our latest addition to our mid-century inspired office furniture collection - the SWC Chair. This chair is part of a beautiful family of furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. With its sleek design and customizable options, the SWC Chair is the perfect choice for any home or business setting.
  </p>
  <p>
    The SWC Chair offers several options for customization. You can choose from a variety of shell colors and base finishes to match your existing decor. The chair is available with plastic back and front upholstery or full upholstery in a range of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. You can also choose whether to have armrests or not, depending on your preference.
  </p>
  <p>
    The construction of the SWC Chair is top-notch. It features a 5-wheel plastic coated aluminum base, ensuring stability and durability. The chair also has a pneumatic adjuster, allowing for easy raise and lower action. This makes it convenient for users to find their desired height and maintain proper posture throughout the day.
  </p>
  <p>
    The SWC Chair is designed with comfort in mind. The seat is made with HD36 foam, providing a plush and supportive cushioning experience. You can also choose between soft or hard-floor caster options, depending on the type of flooring in your space. Additionally, there are two choices of seat foam densities available: medium (1.8 lb/ft3) or high (2.8 lb/ft3). This allows you to customize the level of firmness to suit your preferences.
  </p>
  <p>
    The SWC Chair is not only stylish and comfortable, but it is also built to last. The shell base glider is made of cast aluminum with a modified nylon PA6/PA66 coating, ensuring strength and durability. The shell thickness is 10 mm, providing stability and support. With its high-quality materials and construction, the SWC Chair is qualified for contract use, making it suitable for commercial settings as well.
  </p>
  <p>
    The SWC Chair is proudly made in Italy, known for its craftsmanship and attention to detail. With its timeless design and superior quality, this chair is sure to enhance any space it is placed in.
  </p>
  <h2>Product Dimensions</h2>
  <table>
    <tr>
      <th>Dimension</th>
      <th>Measurement (inches)</th>
    </tr>
    <tr>
      <td>Width</td>
      <td>20.87"</td>
    </tr>
    <tr>
      <td>Depth</td>
      <td>20.08"</td>
    </tr>
    <tr>
      <td>Height</td>
      <td>31.50"</td>
    </tr>
    <tr>
      <td>Seat Height</td>
      <td>17.32"</td>
    </tr>
    <tr>
      <td>Seat Depth</td>
      <td>16.14"</td>
    </tr>
  </table>
</div>

Product IDs: SWC-100, SWC-110
'''

display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""


<div align="center">
  <img src="./Screenshots/Iterative Prompt Engineering.jpg" alt="GNN">
</div>

## **_Summarizing_**

In this lesson, you will summarize text with a focus on specific topics.

### **Setup**

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### **Text to summarize**

In [ ]:
prod_review = """Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there 
might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to her."""

prompt = f"""Your task is to generate a short summary of a product review from an ecommerce site. Summarize the review below, delimited by triple backticks, in at most 30 words. 
Review: ```{prod_review}``` 
"""

response = get_completion(prompt)
print(response)

__Output__

    This panda plush toy is loved by the reviewer's daughter, but they feel it is a bit small for the price.

### **Summarize with a focus on shipping and delivery**

In [ ]:
prompt = f"""Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the Shipping deparmtment. 
Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects that mention shipping and delivery of the product. 
Review: ```{prod_review}``` 
"""

response = get_completion(prompt)
print(response)

__Output__

    The customer is happy with the product but suggests offering larger options for the same price. They were pleased with the early delivery.

### **Summarize with a focus on price and value**

__Comment__: Summaries include topics that are not related to the topic of focus.

In [ ]:
prompt = f"""Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the pricing deparmtment, responsible for determining the price of the product. 
Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects that are relevant to the price and perceived value.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

__Output__

    The customer loves the panda plush toy for its softness and cuteness, but feels it is overpriced compared to other options available.

### **Try "extract" instead of "summarize"**

In [ ]:
prompt = f"""Your task is to extract relevant information from a product review from an ecommerce site to give feedback to the Shipping department. 
From the review below, delimited by triple quotes extract the information relevant to shipping and delivery. Limit to 30 words.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

__Output__

    The shipping department should take note that the product arrived a day earlier than expected.

### **Summarize multiple product reviews**

In [ ]:
# review for toy
review_1 = '''Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there 
might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to her.''' 

# review for a standing lamp
review_2 = """Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price point. Got it fast - arrived in 2 days. The string to the lamp broke during the transit and the company happily 
sent over a new one. Came within a few days as well. It was easy to put together. Then I had a missing part, so I contacted their support and they very quickly got me the missing piece! Seems to me to be a great company that 
cares about their customers and products."""

# review for an electric toothbrush
review_3 = """My dental hygienist recommended an electric toothbrush, which is why I got this. The battery life seems to be pretty impressive so far. After initial charging and leaving the charger plugged in for the first 
week to condition the battery, I've unplugged the charger and been using it for twice daily brushing for the last 3 weeks all on the same charge. But the toothbrush head is too small. I’ve seen baby toothbrushes bigger than 
this one. I wish the head was bigger with different length bristles to get between teeth better because this one doesn't.  Overall if you can get this one around the $50 mark, it's a good deal. The manufactuer's replacements 
heads are pretty expensive, but you can get generic ones that're more reasonably priced. This toothbrush makes me feel like I've been to the dentist every day. My teeth feel sparkly clean!"""

# review for a blender
review_4 = """So, they still had the 17 piece system on seasonal sale for around $49 in the month of November, about half off, but for some reason (call it price gouging) around the second week of December the prices all 
went up to about anywhere from between $70-$89 for the same system. And the 11 piece system went up around $10 or so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part 
where the blade locks into place doesn’t look as good as in previous editions from a few years ago, but I plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the blender first then 
pulverize them in the serving size I want in the blender then switch to the whipping blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade if I need them finer/less 
pulpy). Special tip when making smoothies, finely cut and freeze the fruits and vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making sorbet, use a small to medium sized 
food processor) that you plan to use that way you can avoid adding so much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. I called customer service but the warranty expired 
already, so I had to buy another one. FYI: The overall quality has gone done in these types of products, so they are kind of counting on brand recognition and consumer loyalty to maintain sales. Got it in about two days."""

reviews = [review_1, review_2, review_3, review_4]

for i in range(len(reviews)):
    
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

__Output__

    0 Panda plush toy is loved by daughter, soft and cute, but small for the price. Arrived early. 

    1 Great lamp with storage, fast delivery, excellent customer service, and easy assembly. Highly recommended. 

    2 The reviewer recommends the electric toothbrush for its impressive battery life, but criticizes the small brush head. 

    3 The reviewer found the price increase after the sale disappointing and noticed a decrease in quality. 

## **Inferring**

In this lesson, you will infer sentiment and topics from product reviews and news articles.

### **Setup**

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### **Product review text**

In [ ]:
lamp_review = """Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price point. Got it fast.  The string to our lamp broke during the transit and the company happily sent over a new 
one. Came within a few days as well. It was easy to put together.  I had a missing part, so I contacted their support and they very quickly got me the missing piece! Lumina seems to me to be a great company that cares about 
their customers and products!!"""

### **Sentiment (positive/negative)**

In [ ]:
prompt = f"""What is the sentiment of the following product review, which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    The sentiment of the product review is positive.

In [ ]:
prompt = f"""What is the sentiment of the following product review, which is delimited with triple backticks? Give your answer as a single word, either "positive" or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    positive

### **Identify types of emotions**

In [ ]:
prompt = f"""Identify a list of emotions that the writer of the following review is expressing. Include no more than five items in the list. Format your answer as a list of lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    satisfied, pleased, grateful, impressed, happy

### **Identify anger**

In [ ]:
prompt = f"""Is the writer of the following review expressing anger? The review is delimited with triple backticks. Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    No

### **Extract product and company name from customer reviews**

In [ ]:
prompt = f"""Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with "Item" and "Brand" as the keys. If the information isn't present, use "unknown" as the value. Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    {
    "Item": "lamp",
    "Brand": "Lumina"
    }

### **Doing multiple tasks at once**

In [ ]:
prompt = f"""Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with "Sentiment", "Anger", "Item" and "Brand" as the keys. If the information isn't present, use "unknown" as the value. Make your response 
as short as possible. Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    {
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
    }

### **Inferring topics**

In [ ]:
story = """In a recent survey conducted by the government, public sector employees were asked to rate their level of satisfaction with the department they work at. The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%. 

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top. It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of such an 
innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to hear that our employees are satisfied with their work at NASA. We have a talented and dedicated team who work 
tirelessly to achieve our goals, and it's fantastic to see that their hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction rating, with only 45 percent of employees indicating they were satisfied with their job. The government has pledged to address the 
concerns raised by employees in the survey and work towards improving job satisfaction across all departments."""



### **Infer 5 topics**

In [ ]:
prompt = f"""Determine five topics that are being discussed in the following text, which is delimited by triple backticks. Make each item one or two words long. Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    1. Government survey
    2. Department satisfaction rating
    3. NASA
    4. Social Security Administration
    5. Job satisfaction improvement

In [ ]:
response.split(sep=',')

__Output__

    ['1. Government survey\n2. Department satisfaction rating\n3. NASA\n4. Social Security Administration\n5. Job satisfaction improvement']

### **Make a news alert for certain topics**

In [ ]:
topic_list = ["nasa", "local government", "engineering", "employee satisfaction", "federal government"]

prompt = f"""Determine whether each item in the following list of topics is a topic in the text below, which is delimited with triple backticks. Give your answer as list with 0 or 1 for each topic.

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

__Output__

    [1, 0, 0, 1, 1]

In [ ]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}

if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

__Output__

    ALERT: New NASA story!

## ***Transforming***

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

### **Setup**

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

### **Translation**

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [ ]:
prompt = f"""Translate the following English text to Spanish: ```Hi, I would like to order a blender``` """

response = get_completion(prompt)
print(response)

__Output__

    Hola, me gustaría ordenar una licuadora.

In [ ]:
prompt = f"""Tell me which language this is: ```Combien coûte le lampadaire?``` """

response = get_completion(prompt)
print(response)

__Output__

    This language is French.

In [ ]:
prompt = f"""Translate the following  text to French and Spanish and English pirate: ```I want to order a basketball``` """

response = get_completion(prompt)
print(response)

__Output__

    French: ```Je veux commander un ballon de basket```
    Spanish: ```Quiero ordenar una pelota de baloncesto```
    English: ```I want to order a basketball```

In [ ]:
prompt = f"""Translate the following text to Spanish in both the formal and informal forms: 'Would you like to order a pillow?' """

response = get_completion(prompt)
print(response)

__Output__

    Formal: ¿Le gustaría ordenar una almohada?
    Informal: ¿Te gustaría ordenar una almohada?

### **Universal Translator**

Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [ ]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
]

for issue in user_messages:
    
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""Translate the following text to English and Korean: ```{issue}``` """
    response = get_completion(prompt)
    print(response, "\n")

__Output__

    Original message (The language is French.): La performance du système est plus lente que d'habitude.
    English: "The performance of the system is slower than usual."
    Korean: "시스템의 성능이 평소보다 느립니다."

    Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
    English: "My monitor has pixels that do not light up."
    Korean: "내 모니터에는 밝아지지 않는 픽셀이 있습니다." 

    Original message (The language is Italian.): Il mio mouse non funziona
    English: "My mouse is not working."
    Korean: "내 마우스가 작동하지 않습니다." 

    Original message (The language is Polish.): Mój klawisz Ctrl jest zepsuty
    English: "My Ctrl key is broken"
    Korean: "내 Ctrl 키가 고장 났어요" 

    Original message (The language is Chinese.): 我的屏幕在闪烁
    English: My screen is flickering.
    Korean: 내 화면이 깜박거립니다.

### **Tone Transformation**

Writing can vary based on the intended audience. ChatGPT can produce different tones.

In [ ]:
prompt = f"""Translate the following from slang to a business letter: 'Dude, This is Joe, check out this spec on this standing lamp.' """

response = get_completion(prompt)
print(response)

__Output__

    Dear Sir/Madam,

    I hope this letter finds you well. My name is Joe, and I am writing to bring your attention to a specification document regarding a standing lamp. 

    I kindly request that you take a moment to review the attached document, as it provides detailed information about the features and specifications of the aforementioned standing lamp. 

    Thank you for your time and consideration. I look forward to discussing this matter further with you.

    Yours sincerely,
    Joe

### **Format Conversion**

ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [ ]:
data_json = { "resturant employees" :[
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""Translate the following python dictionary from JSON to an HTML table with column headers and title: {data_json} """

response = get_completion(prompt)
print(response)

In [21]:
response = '''
<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
</head>
<body>

<h2>Restaurant Employees</h2>

<table>
  <tr>
    <th>Name</th>
    <th>Email</th>
  </tr>
  <tr>
    <td>Shyam</td>
    <td>shyamjaiswal@gmail.com</td>
  </tr>
  <tr>
    <td>Bob</td>
    <td>bob32@gmail.com</td>
  </tr>
  <tr>
    <td>Jai</td>
    <td>jai87@gmail.com</td>
  </tr>
</table>

</body>
</html>
'''

In [22]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


### **Spellcheck/Grammar check.**

Here are some examples of common grammar and spelling problems and the LLM's response.

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [ ]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:

    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    
    response = get_completion(prompt)
    print(response)

__Output__

    The girl with the black and white puppies has a ball.
    No errors found.
    It's going to be a long day. Does the car need its oil changed?
    There goes my freedom. They're going to bring their suitcases.
    You're going to need your notebook.
    That medicine affects my ability to sleep. Have you heard of the butterfly effect?
    This phrase is to check chatGPT for spelling ability.

In [ ]:
text = f"""Got this for my daughter for her birthday cuz she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower 
than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price.  It arrived a day earlier than 
expected, so I got to play with it myself before I gave it to my daughter."""

prompt = f"proofread and correct this review: ```{text}```"

response = get_completion(prompt)
print(response)

__Output__

    Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [ ]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

__Output__

    Got this for my daughter for her birthday cuz because she keeps taking mine from my room. room. Yes, adults also like pandas too. too. She takes it everywhere with her, and it's super soft and cute. One cute. However, 
    one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's Additionally, it's a bit small for what I paid for it though. it. I think believe there might be other options 
    that are bigger for the same price. It price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [ ]:
prompt = f"""proofread and correct this review. Make it more compelling. Ensure it follows APA style guide and targets an advanced reader. Output in markdown format. 
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

__Output__

    Review of a Panda Plush Toy: A Perfect Gift for All Ages

    I purchased this delightful panda plush toy as a birthday gift for my daughter, who has a penchant for "borrowing" my belongings from my room. As an adult, I must admit that I too have fallen in love with this adorable 
    creature.

    The first thing that struck me about this plush toy is its exceptional softness. It is an absolute pleasure to touch and hold, making it an irresistible companion for my daughter. She carries it with her everywhere she 
    goes, and it never fails to bring a smile to her face.

    However, I did notice a minor flaw in the design. One of the ears appears to be slightly lower than the other, which I believe was not intended to be asymmetrical. While this does not detract from the overall charm of 
    the toy, it is worth mentioning.

    In terms of size, I must admit that I expected a slightly larger toy considering its price. It would be prudent to explore other options that offer a more substantial size for the same price. Nevertheless, the quality 
    and cuteness of this panda plush toy make it a worthwhile purchase.

    To my surprise, the package arrived a day earlier than expected, allowing me to indulge in some playtime with the toy before presenting it to my daughter. This unexpected bonus added to the overall excitement and 
    anticipation of the gift.

    In conclusion, this panda plush toy is a delightful and endearing gift suitable for individuals of all ages. Its exceptional softness, undeniable cuteness, and the joy it brings make it a compelling choice. While there may 
    be larger options available for the same price, the overall quality and timely delivery make this purchase a satisfying one.

## **_Expanding_**

In this lesson, you will generate customer service emails that are tailored to each customer's review.

### **Setup**

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo",temperature=0): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

### **Customize the automated reply to a customer email**

In [ ]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""So, they still had the 17 piece system on seasonal sale for around $49 in the month of November, about half off, but for some reason (call it price gouging) around the second week of December the prices all went 
up to about anywhere from between $70-$89 for the same system. And the 11 piece system went up around $10 or so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part where 
the blade locks into place doesn't look as good as in previous editions from a few years ago, but I plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the blender first then 
pulverize them in the serving size I want in the blender then switch to the whipping blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade if I need them finer/less 
pulpy). Special tip when making smoothies, finely cut and freeze the fruits and vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making sorbet, use a small to medium sized 
food processor) that you plan to use that way you can avoid adding so much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. I called customer service but the warranty expired 
already, so I had to buy another one. FYI: The overall quality has gone done in these types of products, so they are kind of counting on brand recognition and consumer loyalty to maintain sales. Got it in about two days."""

In [ ]:
prompt = f"""You are a customer service AI assistant. Your task is to send an email reply to a valued customer. Given the customer email delimited by ```, Generate a reply to thank the customer for their review. If the 
sentiment is positive or neutral, thank them for their review. If the sentiment is negative, apologize and suggest that they can reach out to customer service. Make sure to use specific details from the review. Write in a 
concise and professional tone. Sign the email as `AI customer agent`. 

Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

__Output__

    Dear Valued Customer,

    Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

    We are sorry to hear about the price increase you noticed in December. We strive to provide competitive pricing for our products, and we understand your frustration. If you have any further concerns regarding pricing, 
    we recommend reaching out to our customer service team who will be happy to assist you.

    We also appreciate your feedback regarding the base of the system. We continuously work to improve the quality of our products, and your comments will be taken into consideration for future enhancements.

    Regarding the motor issue you encountered, we apologize for any inconvenience caused. Our customer service team is available to assist you with any technical difficulties you may encounter, even if the warranty has expired. 
    Please do not hesitate to reach out to them for further assistance.

    Thank you once again for your review. We value your loyalty and appreciate your support. If you have any further questions or concerns, please feel free to contact our customer service team.

    Best regards,

    AI customer agent

### **Remind the model to use details from the customer's email**

In [ ]:
prompt = f"""You are a customer service AI assistant. Your task is to send an email reply to a valued customer. Given the customer email delimited by ```, Generate a reply to thank the customer for their review. If the 
sentiment is positive or neutral, thank them for their review. If the sentiment is negative, apologize and suggest that they can reach out to customer service. Make sure to use specific details from the review. Write in 
a concise and professional tone. Sign the email as `AI customer agent`.

Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

__Output__

    Dear Valued Customer,

    Thank you for taking the time to share your review with us. We appreciate your feedback and are sorry to hear about your experience.

    We apologize for any inconvenience caused by the price increase you noticed in December. Our pricing is subject to change based on various factors, and we strive to provide competitive prices while maintaining the 
    quality of our products.

    Regarding the issue with the base of the system, we apologize for any disappointment caused. We appreciate your gentleness in handling the blender and understand your concerns about its durability. If you have any 
    further questions or concerns, please feel free to reach out to our customer service team who will be more than happy to assist you.

    We understand your frustration with the motor noise and regret that your warranty had expired at the time of the issue. We always aim to provide reliable products, and we appreciate your loyalty as a customer.

    If there is anything else we can assist you with, please do not hesitate to contact our customer service team. They will be glad to help you further.

    Thank you once again for your review, and we apologize for any inconvenience caused. We value your feedback and will take it into consideration as we continue to improve our products and services.

    Best regards,

    AI customer agent

## ***The Chat Format***

    In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

### **Setup**

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )

    #   print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

__Output__

    Verily, the chicken didst cross the road to locate answers to life's deepest questions. Alas, it hath discovered naught but humorous encounters and jests upon its journey.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

__Output__

    Hello Isa! How can I assist you today?

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

__Output__

    I apologize, but I don't have access to personal information. As a chatbot, I don't have the capability to remember or retrieve personal details.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

__Output__

    Your name is Isa!

### **OrderBot**

We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [ ]:
def collect_messages(_):
    
    prompt = inp.value_input
    inp.value = ''
    
    context.append({'role':'user', 'content':f"{prompt}"})
    
    response = get_completion_from_messages(context) 
    
    context.append({'role':'assistant', 'content':f"{response}"})
    
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content': """
You are OrderBot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then 
summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment. Make sure to clarify all options, extras and sizes to uniquely 
identify the item from the menu. You respond in a short, very conversational friendly style. 

The menu includes:
pepperoni pizza 12.95, 10.00, 7.00 
cheese pizza 10.95, 9.25, 6.50 
eggplant pizza 11.95, 9.75, 6.75 
fries 4.50, 3.50 
greek salad 7.25 

Toppings: 
extra cheese 2.00, 
mushrooms 1.50 
sausage 3.00 
canadian bacon 3.50 
AI sauce 1.50 
peppers 1.00 

Drinks: 
coke 3.00, 2.00, 1.00 
sprite 3.00, 2.00, 1.00 
bottled water 5.00 
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

<div align="center">
  <img src="./Screenshots/Order Bot.jpg">
</div>

__Output__

    Assistant: Hey there! Welcome to our pizza restaurant. How can I assist you today?
    User: Hey! I wanted to order a vegetarian pizza if available..
    Assistant: Of course! We have a delicious vegetarian pizza on our menu. Would you like any specific toppings on it?
    User: Yeah sure. But, does it have a name or something.
    Assistant: Absolutely! Our vegetarian pizza is called the “Garden Delight.” It comes with a variety of fresh vegetables like mushrooms, peppers, onions, and tomatoes. Is there anything else you’d like to add to your order?
    User: Great! I would just want to take that, that’s it!
    Assistant: Perfect! So you’d like to order the “Garden Delight” vegetarian pizza. Is that correct?
    User: Yes please!
    Assistant: Got it! Now, would you like to pick up your order or have it delivered to your address?
    User: I will pick it. How long is it going to take?
    Assistant: Great! The estimated time for pick-up is usually around 20-30 minutes. We’ll make sure your pizza is hot and ready for you when you arrive. Is there anything else I can assist you with?
    User: Nothing else! Just tell me the bill, I shall pay the cash and wait for my delivery!
    Assistant: Sure thing! The total for the “Garden Delight” vegetarian pizza is $11.95. Will you be paying with cash when you pick up your order?
    User: Okay done, thanks!
    Assistant: You’re welcome! Your order has been placed and we’ll have your “Garden Delight” vegetarian pizza ready for pick-up shortly. Thank you for choosing our restaurant and have a great day!

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'''
Create a json summary of the previous food order. Itemize the price for each item. 

The fields should be - 
1) pizza, include size, include price
2) list of toppings 
3) list of drinks, include size, include price
4) list of sides include size, include price
5) total price 
'''
})   

response = get_completion_from_messages(messages, temperature=0)
print(response)

__Output__

    Sure! Here's a JSON summary of your food order:

    {
    "pizza": {
        "size": "12.95",
        "toppings": ["extra cheese", "mushrooms"]
    },
    "drinks": [
        {
        "name": "coke",
        "size": "3.00"
        },
        {
        "name": "sprite",
        "size": "2.00"
        }
    ],
    "sides": [
        {
        "name": "fries",
        "size": "4.50"
        }
    ],
    "total_price": "25.45"
    }

    Please let me know if there's anything else you'd like to add to your order!

<div align="center">
  <img src="./Screenshots/Summary.jpg">
</div>